In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import os.path
import sys
import re
import itertools
import csv
import datetime
import pickle
import random
from collections import defaultdict, Counter
import gc

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import gensim
from sklearn.metrics import f1_score, classification_report, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.sequence import skipgrams
import tensorflow as tf

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

# prepare data
---

In [ ]:
ls -la ../input

In [ ]:
dir_data_src = '../input/anime-recommendations-database'
os.listdir(dir_data_src)

## load anime.csv

In [ ]:
anime_csv = pd.read_csv(os.path.join(dir_data_src, 'anime.csv'))
print(anime_csv.shape)
anime_csv.index = anime_csv.anime_id.values
anime_csv.head()

In [ ]:
np.sort(anime_csv.anime_id.values)

In [ ]:
def Anime_title2id(titlename, csv=anime_csv):
    try:
        return csv.query('name==@titlename').anime_id.values[0]
    except:
        return 9999999

Anime_title2id('Kimi no Na wa.'), Anime_title2id('Fullmetal Alchemist: Brotherhood'), Anime_title2id('Gintama°'), Anime_title2id('PL_ANIME')

In [ ]:
def Anime_id2title(anime_id, csv=anime_csv):
    try:
        return csv.query('anime_id==@anime_id').name.values[0]
    except:
        return '*****'

Anime_id2title(32281), Anime_id2title(5114), Anime_id2title(28977), Anime_id2title(9999999)

## load rating.csv

In [ ]:
'''
load rating.csv
'''
rating_csv = pd.read_csv(os.path.join(dir_data_src, 'rating.csv'))
print(rating_csv.shape)
rating_csv.head()

In [ ]:
'''
`Yuri!!! on Ice` doesn't exist
'''
rating_csv.query('anime_id=="32995"')

In [ ]:
np.unique(rating_csv.user_id.values), len(np.unique(rating_csv.user_id.values))

In [ ]:
np.unique(rating_csv.rating.values)

In [ ]:
rating_csv.rating.values != -1

## create rating_csv2

In [ ]:
rating_csv2 = rating_csv.loc[rating_csv.rating.values != -1]
rating_csv2.shape
rating_csv2.head(10)

## Anime no-rated

In [ ]:
rating_csv2.groupby('anime_id').size().keys().values

In [ ]:
anime_csv.anime_id.values

In [ ]:
np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)

In [ ]:
'''
anime_id no rated
'''
anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]

In [ ]:
'''
how much
'''
anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)].shape

## show anime no-rated

In [ ]:
'''
show anime no rated
'''
anime_id_norated = anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]
print(anime_csv.loc[anime_id_norated].shape)
anime_csv.loc[anime_id_norated]

In [ ]:
rating_csv2.query('anime_id in @anime_id_norated')

## create anime_csv2
delete anime no-rated

In [ ]:
tmp = anime_csv.anime_id.values[np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]
anime_csv2 = anime_csv.loc[tmp].copy()
print(anime_csv2.shape)
anime_csv2

# Load pre-processing data

In [ ]:
from feature_eng import neg_smpl

In [ ]:
src_tfidf_anime = '../input/create-tfidf-of-anime-side'
os.listdir(src_tfidf_anime)

In [ ]:
corpus_csr = scipy.sparse.load_npz(os.path.join(src_tfidf_anime, 'corpus_csr.npz'))
tfidf = gensim.models.TfidfModel.load(os.path.join(src_tfidf_anime, 'tfidf'))
dic_user = gensim.corpora.Dictionary.load(os.path.join(src_tfidf_anime, 'dic_user'))
dic_anime = gensim.corpora.Dictionary.load(os.path.join(src_tfidf_anime, 'dic_anime'))

corpus_csr, tfidf, dic_user, dic_anime

In [ ]:
def Anime2id(title):
    return dic_anime.token2id['anime_id-'+str(Anime_title2id(title))]

Anime2id('Kimi no Na wa.')

# create MatrixSimilarity

In [ ]:
mysim = neg_smpl.MySparseMatrixSimilarity(corpus_csr, num_features=max(dic_user.keys())+1, tfidf=tfidf)

In [ ]:
max(dic_user.keys())

In [ ]:
[
    ['user_id-1', 'user_id-1', 'user_id-2'],
    ['user_id-100', 'user_id-1001', 'user_id-1002'],
]

In [ ]:
dic_user[1]

In [ ]:
dic_user.doc2bow(['user_id-1', 'user_id-1', 'user_id-2'])

In [ ]:
gensim.matutils.corpus2csc([dic_user.doc2bow(['user_id-1', 'user_id-1', 'user_id-2'])], num_terms=mysim.index.shape[1])

In [ ]:
gensim.matutils.corpus2csc([[(0,1),(1,2)], [(1000,1),(69600,2)]], num_terms=mysim.index.shape[1])

# Kimi no Na wa.

In [ ]:
'''
Kimi no Na wa.
'''
mysim.index[Anime2id('Kimi no Na wa.')]

In [ ]:
print(mysim.index.shape)
vec = mysim.index[Anime2id('Kimi no Na wa.')]
corpus = gensim.matutils.scipy2sparse(vec)
corpus[:20]

# this 'corpus' is user rating of 'Kimi no Na wa.'

In [ ]:
'''
this `corpus` is user rating of `Kimi no Na wa.`
'''
for userid, rating in itertools.islice(corpus, 10):
    print(dic_user[userid], rating)

In [ ]:
'''
calc similar anime to `corpus`
'''
print('Kimi no Na wa. id =', Anime2id('Kimi no Na wa.'))
mysim.num_best = 10
res = mysim[corpus]
res

In [ ]:
'''
`res` is similar animes to `corpus`
'''
for idx, wgt in res:
    print(wgt, Anime_id2title(int(re.sub('anime_id-', '', dic_anime[idx]))))

# Mobile Suit Gundam

In [ ]:
'''
get user rating
'''
vec = mysim.index[Anime2id('Mobile Suit Gundam')]
corpus = gensim.matutils.scipy2sparse(vec)
corpus[:15]

In [ ]:
%%time
'''
calc similar anime to `corpus`
'''
mysim.num_best = 10
res = mysim[corpus]
res

In [ ]:
'''
`res` is similar animes to `corpus`
'''
for idx, wgt in res:
    print(wgt, Anime_id2title(int(re.sub('anime_id-', '', dic_anime[idx]))))

In [ ]:
%%time
'''
method = 'WT_TFIDF'
'''
mysim.num_best = 10
mysim.method = 'WT_TFIDF'
res = mysim[corpus]
res

In [ ]:
'''
`res` is similar animes to `corpus`
'''
for idx, wgt in res:
    print(wgt, Anime_id2title(int(re.sub('anime_id-', '', dic_anime[idx]))))